# W&H Data Analysis Presentation

In [15]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns 
from sklearn.linear_model import LinearRegression
import IPython.core.display as di
from IPython.display import HTML
import pickle
from collections import OrderedDict
import collections
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
df = pd.read_excel('PRIMARY WAGE AND HOUR SUB TRACKING.xls',skiprows=2)
df = df.dropna(subset = ['Actuarial Pricing Model', 'Price Indicated'])
df = df[~df['Actuarial Pricing Model'].isin([' ','DNP'])]
df = df[df['Actuarial Pricing Model']!=0]
df = df[~df['Price Indicated'].isin([' '])]
df['# of Ees'] = df['# of Ees'].fillna(df['# of Ees'].mean())
df['Loss History '] = df['Loss History '].fillna(0)
df['Price Difference'] = df['Actuarial Pricing Model'] - df['Price Indicated']
df['Price Difference Percentage'] = df['Price Difference']/df['Actuarial Pricing Model']*100

## Let's see the first 5 rows of the data

In [3]:
df.head(5)

,Insured,Insured_Count,Received,Broker,Contact,Industry,# of Ees,Limit ($M),SIR,Actuarial Pricing Model,...,Loss History,Price Indicated,RPM.1,Date indicated,Call Y/N,Quote Date,Comments,Glassdoor,Price Difference,Price Difference Percentage
1,OTO Development,1,2017-04-20,JLT,Ian,Hospitality,3119.482014,10.0,0.75,213790,...,0.0,96000,9600,NaN,NaN,NaT,NaN,2.9,117790,55.0961
2,Dutch Fashions,1,2017-05-05,AON,Darren,Clothing & Clothing Accessories,3119.482014,5.0,0.50,46705,...,0.0,90000,18000,2017-05-18 00:00:00,NaN,NaT,NaN,NaN,-43295,-92.6989
3,Moravia Holdings,1,2017-05-08,JLT,Vernae,"Professional, Scientific and Tech..",3119.482014,1.0,0.50,77567,...,0.0,77567,77567,2017-05-17 00:00:00,NaN,NaT,Client decided not to purchase at this time.,3.3,0,0
4,Moravia Holdings,0,2017-05-08,JLT,Vernae,"Professional, Scientific and Tech..",3119.482014,2.0,0.50,108471,...,0.0,108471,54235.5,2017-05-17 00:00:00,NaN,NaT,NaN,3.3,0,0
5,Moravia Holdings,0,2017-05-08,JLT,Vernae,"Professional, Scientific and Tech..",3119.482014,3.0,0.50,121362,...,0.0,121362,40454,2017-05-17 00:00:00,NaN,NaT,NaN,3.3,0,0


Note: In this analysis, for some plots we remove outliers so the plots reflect the data better.

In [4]:
df = df.dropna(subset=['# of Ees'])
dummy_industry = pd.get_dummies(df['Industry'])
dummy_broker = pd.get_dummies(df['Broker'])
df = pd.concat([df,dummy_industry,dummy_broker],axis=1)

## Visualization of the variables

In [5]:
apm, apm_ax = plt.subplots(figsize=(10, 10))
sns.distplot(df["Actuarial Pricing Model"].as_matrix(), kde=False);
apm_ax.set_xlabel("Actuarial Model Prices",fontsize=14)
plt.close()

In [6]:
def plot(varname=''):
    import time
    fig, ax = plt.subplots(figsize=(30, 20))
    if varname == 'Actuarial Pricing Model Plot':
        fig = sns.distplot(df["Actuarial Pricing Model"].as_matrix(), kde=False);
        ax.set_xlabel("Actuarial Model Prices",fontsize=40)
        plt.yticks(fontsize=30)
        plt.xticks(fontsize=30)
        print('The Average of Actuarial Model Prices is: {}'.format(df["Actuarial Pricing Model"].as_matrix().mean()))
        print('The Standard Deviation of Actuarial Model Prices is: {}'.format(df["Actuarial Pricing Model"].as_matrix().std()))
        
    elif varname == 'Indicated Price Plot':
        fig = sns.distplot(df["Price Indicated"].as_matrix(), kde=False);
        ax.set_xlabel("Indicated Prices",fontsize=40)
        plt.yticks(fontsize=30)
        plt.xticks(fontsize=30)
        print('The Average of Indicated Prices is: {}'.format(df["Price Indicated"].as_matrix().mean()))
        print('The Standard Deviation of Indicated Prices is: {}'.format(df["Price Indicated"].as_matrix().std()))
        
    elif varname =='RPM Plot':
        fig = sns.distplot(df["RPM"].as_matrix(), kde=False);
        ax.set_xlabel("RPM",fontsize=40)
        plt.yticks(fontsize=30)
        plt.xticks(fontsize=30)
        print('The Average of RPM is: {}'.format(df["RPM"].as_matrix().mean()))
        print('The Standard Deviation of RPM is: {}'.format(df["RPM"].as_matrix().std()))
        
    elif varname == 'Price Difference Plot':
        fig = sns.distplot(df["Price Difference"].as_matrix(), kde=False);
        ax.set_xlabel("Price Difference",fontsize=40)
        plt.yticks(fontsize=30)
        plt.xticks(fontsize=30)
        print('The Average of Price Difference is: {}'.format(df["Price Difference"].as_matrix().mean()))
        print('The Standard Deviation of RPM is: {}'.format(df["Price Difference"].as_matrix().std()))
        
    elif varname == 'Number of Employees Plot':
        fig = sns.distplot(df["# of Ees"].dropna().as_matrix(), kde=False);
        ax.set_xlabel("Number of employees",fontsize=40)
        plt.yticks(fontsize=30)
        plt.xticks(fontsize=30)
        print('The Average of Number of Employees is: {}'.format(df["# of Ees"].as_matrix().mean()))
        print('The Standard Deviation of Number of Employees is: {}'.format(df["# of Ees"].as_matrix().std()))
        
    elif varname == 'Insured Count Plot':
        plt.close()
        fig, ax = plt.subplots(figsize=(30, 30))
        fig = sns.countplot(y=df["Insured"],order=df["Insured"].value_counts().index)
        plt.yticks(fontsize=25)
        plt.xticks(fontsize=25)
        ax.set_xlabel('Counts',fontsize=40)
        ax.set_ylabel('Insured',fontsize=40)  
        
    elif varname == 'Industry Count Plot':
        fig = sns.countplot(y=df["Industry"],order=df["Industry"].value_counts().index)
        plt.yticks(fontsize=30)
        plt.xticks(fontsize=30)
        ax.set_xlabel('Counts',fontsize=40)
        ax.set_ylabel('Industry',fontsize=40)    
        
    elif varname == 'Broker Count Plot':
        fig=sns.countplot(y=df["Broker"],order=df["Broker"].value_counts().index)
        plt.yticks(fontsize=30)
        plt.xticks(fontsize=30)
        ax.set_xlabel('Counts',fontsize=40)
        ax.set_ylabel('Broker',fontsize=40)
        
    elif varname == 'Loss History Plot':
        fig = sns.countplot(y=df["Loss History "].dropna().as_matrix())
        plt.yticks(fontsize=40)
        plt.xticks(fontsize=40)
        ax.set_xlabel('Loss History Number',fontsize=30)
        ax.set_ylabel('Counts',fontsize=30)
        
    plt.show()        

In [7]:
widget = widgets.Dropdown(
    options=['Actuarial Pricing Model Plot','Indicated Price Plot','RPM Plot','Price Difference Plot',
             'Number of Employees Plot', 'Insured Count Plot', 'Industry Count Plot','Broker Count Plot',
             'Loss History Plot'
            ],
    description='Choose a plot to view:',
    disabled=False,
)
tool = widgets.interactive(plot, varname=widget)
display(tool)

Widget Javascript not detected.  It may not be installed or enabled properly.


## Analysis of the relationships between the variables and price difference

### Price Difference Analysis

In [8]:
def plot(varname=''):
    import time
    fig, ax = plt.subplots(figsize=(30, 20))
    if varname == 'Industry and Price Difference':
        industry_list = df['Industry'].dropna().unique()
        relationship_dict = {}
        for i in range(0,len(industry_list)):
            try:
                tmp_df = df[df['Industry'] == industry_list[i]] 
                percent_difference = tmp_df['Price Difference Percentage'].mean()*100
            except Exception as e:
                pass
            relationship_dict[industry_list[i]] = percent_difference
        relationship_dict = OrderedDict(sorted(relationship_dict.items(), key=lambda x: x[1]))
        for industry,percent in relationship_dict.items():
            print("{}: {:3.1f}% price difference".format(industry,percent))
        del relationship_dict['Utilities']
        X = np.arange(len(relationship_dict))
        fig = plt.barh(X, relationship_dict.values(), align='center')
        plt.yticks(X, list(relationship_dict.keys()),fontsize=30)
        ax.set_xlabel("Percentage Price Difference",fontsize=40)
        ax.set_ylabel("Industry",fontsize=40)
        plt.xticks(fontsize=30)
        print('Note: We remove extreme outliers from the plot to get a better scale')
    
    elif varname == 'Overcharge Analysis':
        industry_list = df['Industry'].dropna().unique()
        relationship_dict = {}
        for i in range(0,len(industry_list)):
            tmp_df = df[df['Industry'] == industry_list[i]] 
            tmp_df = tmp_df[tmp_df['Price Difference Percentage']>0]
            tmp = tmp_df['Price Difference Percentage'].values
            if len(tmp) > 0:
                overcharge_percentage = tmp.mean()
            else:
                overcharge_percentage = 0
            relationship_dict[industry_list[i]] = overcharge_percentage
        relationship_dict = OrderedDict(sorted(relationship_dict.items(), key=lambda x: x[1]))
        X = np.arange(len(relationship_dict))
        fig = plt.barh(X, relationship_dict.values(), align='center')
        plt.yticks(X, list(relationship_dict.keys()),fontsize=30)
        ax.set_xlabel("Overcharge Percentage",fontsize=40)
        ax.set_ylabel("Industry",fontsize=40)
        plt.xticks(fontsize=30)
        
    elif varname == 'Undercharge Analysis':
        industry_list = df['Industry'].dropna().unique()
        relationship_dict = {}
        for i in range(0,len(industry_list)):
            tmp_df = df[df['Industry'] == industry_list[i]] 
            tmp_df = tmp_df[tmp_df['Price Difference Percentage']<0]
            tmp = tmp_df['Price Difference Percentage'].values
            if len(tmp) > 0:
                overcharge_percentage = tmp.mean()
            else:
                overcharge_percentage = 0
            relationship_dict[industry_list[i]] = overcharge_percentage
        relationship_dict = OrderedDict(sorted(relationship_dict.items(), key=lambda x: x[1]))
        del relationship_dict['Utilities']
        X = np.arange(len(relationship_dict))
        fig = plt.barh(X, relationship_dict.values(), align='center')
        plt.yticks(X, list(relationship_dict.keys()),fontsize=30)
        ax.set_xlabel("Undercharge Percentage",fontsize=40)
        ax.set_ylabel("Industry",fontsize=40)
        plt.xticks(fontsize=30)
        
    elif varname == 'Overcharge vs Undercharge vs Correct Price List':
        plt.close()
        industry_list = df['Industry'].dropna().unique()
        relationship_dict = {}
        for i in range(0,len(industry_list)):
            try:
                tmp_df = df[df['Industry'] == industry_list[i]] 
                undercharge_count = 0
                overcharge_count = 0
                correctly_count = 0
                tmp = tmp_df['Price Difference'].values
                for j in range(0,len(tmp)):
                    if tmp[j] < 0:
                        undercharge_count += 1
                    elif tmp[j] > 0:
                        overcharge_count += 1
                    else:
                        correctly_count +=1
            except Exception as e:
                undercharge_percentage = 0
                pass
            print("{} overcharge/undercharge counts: ".format(industry_list[i]))
            print("Overcharge: {}".format(overcharge_count))
            print("Undercharge: {}".format(undercharge_count))
            print("Correctly charge: {}".format(correctly_count))
            print('==========================')
            relationship_dict[industry_list[i]] = (overcharge_count,undercharge_count,correctly_count)
        
    elif varname == 'Number of Employees vs Price Difference Analysis':
        tmp_df = df[["# of Ees","Price Difference Percentage",'Industry']].dropna()
        tmp_df = tmp_df[tmp_df['Industry']!='Utilities']
        fig = sns.regplot(x=tmp_df['# of Ees'],y=tmp_df['Price Difference Percentage'])
        plt.yticks(fontsize=30)
        ax.set_xlabel("Number of Employees",fontsize=40)
        ax.set_ylabel("Price Difference Percentage",fontsize=40)
        plt.xticks(fontsize=30)
        print("We can see that number of employees is positively correlated with Price Difference Percentage")
        relationship_dict = collections.OrderedDict()
        for i in range(1,13):
            try:
                tmp_df = df[df['# of Ees'] < 1000*i] 
                tmp_df = tmp_df[tmp_df['Price Difference Percentage'] >-300]
                tmp_df = tmp_df[tmp_df['Price Difference Percentage'] < 300]
                tmp_df = tmp_df[tmp_df['# of Ees'] > 1000*(i-1)]
                percent_difference = tmp_df['Price Difference Percentage'].mean()
            except Exception as e:
                percent_difference = 0
                pass
            relationship_dict[ str((i-1)*1000) + " to " + str(i*1000) + " employees"] = percent_difference
        fig1, ax1 = plt.subplots(figsize=(30, 20))
        X = np.arange(len(relationship_dict))
        fig1 = plt.barh(X, relationship_dict.values(), align='center')
        plt.yticks(X, list(relationship_dict.keys()),fontsize=30)
        plt.xticks(fontsize=30)
        ax1.set_xlabel("Percentage Price Difference",fontsize=40)
        ax1.set_ylabel("Number of Employees",fontsize=40)
    
    elif varname == 'Loss History vs Price Difference Analysis':
        tmp_df = df[["Loss History ","Price Difference Percentage",'Industry']].dropna()
        tmp_df = tmp_df[tmp_df['Industry']!='Utilities']
        fig = sns.regplot(x=tmp_df["Loss History "],y=tmp_df['Price Difference Percentage'])
        plt.yticks(fontsize=30)
        ax.set_xlabel("Loss History",fontsize=40)
        ax.set_ylabel("Price Difference Percentage",fontsize=40)
        plt.xticks(fontsize=30)
        relationship_dict = collections.OrderedDict()
        for i in range(0,4):
            tmp_df = df[df['Loss History '] == i] 
            tmp_df = tmp_df[tmp_df['Price Difference Percentage'] >-300]
            tmp_df = tmp_df[tmp_df['Price Difference Percentage'] < 300]
            percent_difference = tmp_df['Price Difference Percentage'].mean()
            relationship_dict[ str(i) + " Loss history"] = percent_difference
        fig1, ax1 = plt.subplots(figsize=(30, 20))
        X = np.arange(len(relationship_dict))
        fig1 = plt.barh(X, relationship_dict.values(), align='center')
        plt.yticks(X, list(relationship_dict.keys()),fontsize=30)
        plt.xticks(fontsize=30)
        ax1.set_xlabel("Percentage Price Difference",fontsize=40)
        ax1.set_ylabel("Loss History",fontsize=40)
        
    plt.show()        

In [9]:
widget = widgets.Dropdown(
    options=['Industry and Price Difference','Overcharge Analysis','Undercharge Analysis',
             'Overcharge vs Undercharge vs Correct Price List', 
             'Number of Employees vs Price Difference Analysis',
             'Loss History vs Price Difference Analysis'
            ],
    description='Choose an analysis:',
    disabled=False,
)
tool = widgets.interactive(plot, varname=widget)
display(tool)

Widget Javascript not detected.  It may not be installed or enabled properly.


## Analysis of Glassdoor ratings and its relationship with RPM and Loss History

In [10]:
from collections import OrderedDict
def plot(varname=''):
    import time
    fig, ax = plt.subplots(figsize=(30, 20))
    if varname == 'Glassdoor Rating Distribution':
        fig = sns.distplot(df["Glassdoor"].dropna().as_matrix(), kde=False);
        ax.set_xlabel("Glassdoor Rating",fontsize=40)
        plt.yticks(fontsize=30)
        plt.xticks(fontsize=30)
        print('The Average of Glassdoor Rating is: {}'.format(df['Glassdoor'].dropna().as_matrix().mean()))
        print('The Standard Deviation of Glassdoor Rating is: {}'.format(df["Glassdoor"].dropna().as_matrix().std()))
        
    elif varname == 'Glassdoor Rating vs RPM':
        tmp_df = df[["Glassdoor","RPM",'Industry']].dropna()
        tmp_df = tmp_df[tmp_df['Industry']!='Utilities']
        fig = sns.regplot(x=tmp_df['Glassdoor'],y=tmp_df['RPM'])
        plt.yticks(fontsize=30)
        ax.set_xlabel("Glassdoor Rating",fontsize=40)
        ax.set_ylabel("RPM",fontsize=40)
        plt.xticks(fontsize=30)
        
    elif varname == 'Glassdoor Rating vs Loss History':
        tmp_df = df[["Glassdoor","Loss History ",'Industry']].dropna()
        tmp_df = tmp_df[tmp_df['Industry']!='Utilities']
        fig = sns.regplot(x=tmp_df['Glassdoor'],y=tmp_df['Loss History '])
        plt.yticks(fontsize=30)
        ax.set_xlabel("Glassdoor Rating",fontsize=40)
        ax.set_ylabel("Loss History",fontsize=40)
        plt.xticks(fontsize=30)
    plt.show()        

In [11]:
widget = widgets.Dropdown(
    options=['Glassdoor Rating Distribution','Glassdoor Rating vs RPM','Glassdoor Rating vs Loss History'
            ],
    description='Choose an analysis:',
    disabled=False,
)
tool = widgets.interactive(plot, varname=widget)
display(tool)

Widget Javascript not detected.  It may not be installed or enabled properly.


## Regression Analysis

In [12]:
from collections import OrderedDict
def plot(varname=''):
    tmp_df = df.dropna(subset=['Glassdoor'])
    variables = ['# of Ees','Loss History ','Accommodation',
         'Administrative, Support…',
         'Agriculture, Forestry, Fishing & Hunting',
         'Ambulatory Health Care', 'Arts, Entertainment & Recreation',
         'Clothing & Clothing Accessories', 'Finance & Insurance',
         'Finance & Insurance (Healthcare)', 'Finance and Insurance',
         'Food Service', 'Food Services and Drinking Places',
         'Food and Beverage Stores', 'Healthcare', 'Hospitality',
         'Hospitals', 'Information', 'Manufacturing',
         'Motor Vehicle and Parts Dealers', 'Motor Vehicles & Parts',
         'Nursing & Residential Care Facilities',
         'Professional, Scientific and Tech..',
         'Real Estate and Rental and Leasing', 'Repair & Maintenance',
         'Retail', 'Transportation and Warehousing', 'Utilities',
         'Wholesale Trade', 'AON', 'Arch London', 'BESSO', 'Beecher ',
         'JLT', 'Marsh', 'Paragon', 'Willis','Glassdoor']
    tmp_df = tmp_df[tmp_df['Price Difference Percentage'] >-200]
    tmp_df = tmp_df[tmp_df['Price Difference Percentage'] < 200]
    x =  tmp_df[variables]
    y = tmp_df['Price Difference Percentage'].as_matrix().reshape(-1,1)
    if varname == 'Regression of all variables vs Price Difference Percentage':
        lin_reg = LinearRegression(fit_intercept=False,normalize=True)
        lin_reg.fit(x,y)
        for i in range(0,len(variables)):
            print('Coefficients of variable {}: {:3.1f}'.format(variables[i],lin_reg.coef_[0][i]))
    elif varname == 'Regularized regression of all variables vs Price Difference Percentage':
        enet_reg = ElasticNet(fit_intercept=False,normalize=True,alpha=0.2, l1_ratio=0.3)
        enet_reg.fit(x,y)
        enet_reg.score(x,y)
        for i in range(0,len(variables)):
            print('Coefficients of variable {}: {:3.1f}'.format(variables[i],enet_reg.coef_[i]))

In [13]:
widget = widgets.Dropdown(
    options=['Regression of all variables vs Price Difference Percentage',
             'Regularized regression of all variables vs Price Difference Percentage'
            ],
    description='Choose an analysis:',
    disabled=False,
)
tool = widgets.interactive(plot, varname=widget)
display(tool)

Widget Javascript not detected.  It may not be installed or enabled properly.
